In [1]:
import re
import nltk
import string
import pickle
import numpy as np
import pandas as pd
import simplejson as sj
from nltk.collocations import *
from collections import defaultdict
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
np.random.seed(1738)

In [2]:
import set_sent_model
reload(set_sent_model)
from set_sent_model import *

###Import the data and split it into training and testing

In [8]:
d_path = 'yds/yelp_academic_dataset_review.json'
original_data = get_data(d_path)
train, test = train_test_split(original_data, test_size=0.3)

###Create word features (vocabulary)¶

In [9]:
p_data = [doc for doc in train if doc['stars'] == 5]
n_data = [doc for doc in train if doc['stars'] == 1]

filtered = []
stop_words = nltk.corpus.stopwords.words('english')
punct = string.punctuation

filtered.extend(stop_words)
filtered.extend(punct)
filtered.extend(["...", "''", "``"])

for doc in p_data:
    text = doc['text'].lower().encode('ascii', 'ignore')
    words = nltk.word_tokenize(text)
    words = [w for w in words if w not in filtered]
    doc['words'] = words

for doc in n_data:
    text = doc['text'].lower().encode('ascii', 'ignore')
    words = nltk.word_tokenize(text)
    words = [w for w in words if w not in filtered]
    doc['words'] = words

In [10]:
all_words = []

[all_words.append(w) for doc in p_data for w in doc['words']]
[all_words.append(w) for doc in n_data for w in doc['words']]

all_words = nltk.FreqDist(all_words)
all_words = all_words.most_common(5000)

word_features = [tup[0] for tup in all_words]

In [12]:
with open('algos/word_features.pickle', 'wb') as f:
    pickle.dump(word_features, f)

###Create feature matrix from word features

In [3]:
d_path = 'yds/yelp_academic_dataset_review.json'
original_data = get_data(d_path)
train, test = train_test_split(original_data, test_size=0.3)

In [4]:
datadf = pd.DataFrame(train)
X = datadf[datadf.stars != 3]
y = X.apply(lambda x: 1 if x.stars > 3 else 0, axis=1)

In [5]:
with open('algos/word_features.pickle', 'rb') as f:
    word_features = pickle.load(f)

In [6]:
for w in word_features[:400]: #6ish hours
    X[w] = X.apply(lambda x: 1 if w in x.text else 0, axis=1)

/Users/jachian/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [7]:
X.drop(['categories', 'review_id', 'date', 'business_id', 'text', 'type', 'user_id', 'votes', 'stars'], axis=1, inplace=True)

/Users/jachian/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
testdf = pd.DataFrame(test)
X_test = testdf[testdf.stars != 3]
y_test = X_test.apply(lambda x: 1 if x.stars > 3 else 0, axis=1)
for w in word_features[:400]:
    X_test[w] = X_test.apply(lambda x: 1 if w in x.text else 0, axis=1)
X_test.drop(['categories', 'review_id', 'date', 'business_id', 'text', 'type', 'user_id', 'votes', 'stars'], axis=1, inplace=True)

/Users/jachian/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jachian/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
d_path = 'yds/yelp_academic_dataset_review.json'
original_data = get_data(d_path)
train, test = train_test_split(original_data, test_size=0.3)
with open('algos/word_features.pickle', 'rb') as f:
    word_features = pickle.load(f)
X, y, X_test, y_test = create_feature_matrix(train, test, word_features[:400], mapped=False)

Preliminary sentiment prediction model using random word features from the top 5000 most frequently used words

In [9]:
rf = RandomForestClassifier()
rf.fit(X, y)
rf.score(X_test, y_test)

0.85594729273801318

In [10]:
with open('algos/RandomForestClassifier.pickle', 'wb') as f:
    pickle.dump(rf, f)

Most important features from preliminary model

In [ ]:
top_100 = np.argsort(rf.feature_importances_)[-1:-101:-1]
X_test.columns[top_100]

In [11]:
lr = LogisticRegression()
lr.fit(X, y)
lr.score(X_test, y_test)

0.88914426382310885

In [12]:
with open('algos/LogRegressionClassifier.pickle', 'wb') as f:
    pickle.dump(lr, f)

In [ ]:
top_100 = np.argsort(lr.coef_[0])[-1:-101:-1]
X_test.columns[top_100]

In [ ]:
sorted(lr.coef_[0])[-1:-101:-1]

###Exploratory Analysis of Combinations and Markhov Chains

In [ ]:
'''
Number
Tag
Description
1.	CC	Coordinating conjunction
2.	CD	Cardinal number
3.	DT	Determiner
4.	EX	Existential there
5.	FW	Foreign word
6.	IN	Preposition or subordinating conjunction
7.	JJ	Adjective
8.	JJR	Adjective, comparative
9.	JJS	Adjective, superlative
10.	LS	List item marker
11.	MD	Modal
12.	NN	Noun, singular or mass
13.	NNS	Noun, plural
14.	NNP	Proper noun, singular
15.	NNPS	Proper noun, plural
16.	PDT	Predeterminer
17.	POS	Possessive ending
18.	PRP	Personal pronoun
19.	PRP$	Possessive pronoun
20.	RB	Adverb
21.	RBR	Adverb, comparative
22.	RBS	Adverb, superlative
23.	RP	Particle
24.	SYM	Symbol
25.	TO	to
26.	UH	Interjection
27.	VB	Verb, base form
28.	VBD	Verb, past tense
29.	VBG	Verb, gerund or present participle
30.	VBN	Verb, past participle
31.	VBP	Verb, non-3rd person singular present
32.	VBZ	Verb, 3rd person singular present
33.	WDT	Wh-determiner
34.	WP	Wh-pronoun
35.	WP$	Possessive wh-pronoun
36.	WRB	Wh-adverb
'''

In [22]:
d_path = 'yds/yelp_academic_dataset_review.json'
w_exploration = get_data(d_path)
train_w, test_w = train_test_split(w_exploration, test_size=0.3)

train_w = [rev['text'].lower() for rev in train_w]

In [23]:
train_w = [nltk.word_tokenize(rev) for rev in train_w]

In [26]:
pre, post = exploratory_analysis(train_w, word_features[:1000])

In [ ]:
print pre['wonderful']
print post['wonderful']
print pre['fantastic']
print post['fantastic']

In [ ]:
print pre['burger'], '\n'
print pre['restaurant'], '\n'
print pre['chicken'], '\n'
print post['chicken'], '\n'
print pre['meal'], '\n'

In [ ]:
for w in [w for w in nltk.pos_tag(word_features[:400]) if 'JJ' in w[1]]:
    print w

In [83]:
print pre['friendly']
print post['friendly']

[('very', 17010), ('and', 13070), ('was', 9194), (',', 5983), ('super', 5362), ('is', 5146), ('.', 2937), ('always', 2560), ('are', 2548), ('a', 2125)]
[('and', 28901), ('.', 17967), (',', 13242), ('staff', 6888), ('service', 5306), ('!', 1587), ('but', 1361), ('as', 791), ('people', 671), (u'&', 613)]


In [ ]:
for w in [w for w in nltk.pos_tag(word_features[:400]) if 'NN' in w[1]]:
    print w

In [44]:
nltk.pos_tag(['really']) #RB RBR RBS
# really, very, even, much, real?

[('really', 'RB')]

Explore mapping subjects to:
- place
- service
- experience
- prices
- price
- portions
- money
- style
- decor
- serve
- cheap

In [44]:
for k, w in enumerate(word_features[:1000]):
    if 'price' in w:
        print k, w
    if 'money' in w:
        print k, w
print ''
        
for key in ['price', 'prices', 'money', 'priced', 'pricey', 'overpriced']:
    print key.upper()
    print pre[key],  '\n'
print 'CHEAP'
print pre['cheap']
print post['cheap']

133 price
158 prices
299 money
551 priced
709 pricey
942 overpriced

PRICE
[('the', 30428), ('.', 2382), ('reasonable', 2251), (u'great', 2060), ('good', 1830), ('same', 1772), ('half', 1226), ('full', 1217), ('that', 1182), ('and', 1043), ('decent', 899), ('a', 691), ('this', 620), (',', 587), ('1/2', 528), ('fair', 523), ('in', 502), ('high', 460), ('low', 456), ('affordable', 345)] 

PRICES
[('the', 19213), ('.', 5359), ('reasonable', 3012), ('great', 2248), ('their', 1819), ('good', 1743), ('and', 1666), (',', 1090), ('hour', 1017), ('high', 869), ('decent', 855), ('!', 607), ('drink', 535), ('cheap', 487), ('low', 448), ('fair', 445), ('these', 393), ('affordable', 314), ('menu', 295), ('lunch', 287)] 

MONEY
[('the', 4490), ('your', 3885), ('my', 2602), ('of', 2240), ('and', 790), ('our', 722), ('more', 671), ('much', 586), ('some', 556), ('for', 463), ('or', 393), ('their', 352), ('save', 338), ('less', 314), ('spend', 261), ('extra', 252), ('earned', 237), ('spending', 208), ('

In [ ]:
for k, w in enumerate(word_features[:1000]):
    if 'service' in w:
        print k, w
    if 'serve' in w:
        print k, w
print ''

for key in ['service', 'customer', 'server', 'served', 'serve', 'servers']:
    print key.upper()
    print pre[key], '\n'
    print post[key], '\n'

In [ ]:
for k, w in enumerate(word_features[:1000]):
    if 'decor' in w:
        print k, w
    if 'ambiance' in w:
        print k, w
    if 'atmosphere' in w:
        print k, w
print ''

for key in ['ambiance', 'decor', 'atmosphere']:
    print key.upper()
    print pre[key], '\n'
    print post[key], '\n'

In [ ]:
for key in ['experience', 'place', 'portions']:
    print key.upper()
    print pre[key], '\n'
    print post[key], '\n'

###General Notes
- good, great, bad, worst, etc. map well to subjects that follow
- joining "over" and "reasonably" to "priced" may create a nice feature
- may be worth mapping "high/low price", "high/low prices" together
- eliminate "customer" to better map the word "service"

###Subject mapping notes
- "server", "servers", and "service" for service subject mappers
- "decor", and "ambiance" for decor subject mappers
- "price(s)" for price point mappers

### Current order for extra preprocessing steps
- join contractions
- remove amplifier adverbs and "customer" (really, very, even, much, customer)
- join sentiment flippers (not)
- join "priced" (over, reasonably)
- map generic adjectives to their following subjects (good, bad, worst, etc.)

Now we try adjusting the processing steps after tokenizing the documents by removing adverbs from the list of words, then combining the word "not" to its following word.

In [3]:
d_path = 'yds/yelp_academic_dataset_review.json'
original_data2 = get_data(d_path)
train2, test2 = train_test_split(original_data2, test_size=0.3)

In [4]:
filtered = ['really', 'very', 'even', 'much', 'real', 'customer', 'the', \
           'our', 'my', 'is', 'was']
subjects = ['service', 'ambiance', 'portions', 'food', 'decor', 'price', \
           'restaurant', 'experience']

In [5]:
for rev in train2:
    text = rev['text'].lower()
    text = text.encode('ascii', 'ignore')
    words = nltk.word_tokenize(text)
    words = [w for w in words if w not in filtered]
    words = mapper(words)
    words = join_fol(words, ['not'])
    words = join_pre(words, ['priced'])
    words = re_position(words)
    words = join_pre(words, subjects)
    words = [w for w in words if 'restaurant' not in w and 'experience' not in w]
    rev['words'] = words

In [6]:
for rev in test2:
    text = rev['text'].lower()
    text = text.encode('ascii', 'ignore')
    words = nltk.word_tokenize(text)
    words = [w for w in words if w not in filtered]
    words = mapper(words)
    words = join_fol(words, ['not'])
    words = join_pre(words, ['priced'])
    words = re_position(words)
    words = join_pre(words, subjects)
    words = [w for w in words if 'restaurant' not in w and 'experience' not in w]
    rev['words'] = words

In [7]:
with open('algos/word_feats1.pickle', 'rb') as f:
    word_feats = pickle.load(f)

In [8]:
datadf2 = pd.DataFrame(train2) 
X2 = datadf2[datadf2.stars != 3]
y2 = X2.apply(lambda x: 1 if x.stars > 3 else 0, axis=1)

In [9]:
for w in word_feats:
    X2[w] = X2.apply(lambda x: 1 if w in x.words else 0, axis=1)

/Users/jachian/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [10]:
X2.drop(['words', 'categories', 'review_id', 'date', 'business_id', 'text', 'type', 'user_id', 'votes', 'stars'], axis=1, inplace=True)

/Users/jachian/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
testdf = pd.DataFrame(test2)
X_test2 = testdf[testdf.stars != 3]
y_test2 = X_test2.apply(lambda x: 1 if x.stars > 3 else 0, axis=1)
for w in word_feats:
    X_test2[w] = X_test2.apply(lambda x: 1 if w in x.words else 0, axis=1)
X_test2.drop(['words', 'categories', 'review_id', 'date', 'business_id', 'text', 'type', 'user_id', 'votes', 'stars'], axis=1, inplace=True)

/Users/jachian/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jachian/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
train, test = map_word_feats(train, test)
with open('algos/word_feats1.pickle', 'rb') as f:
    mapped_feats = pickle.load(f)
X2, y2, X_test2, y_test2 = create_feature_matrix(train, test, mapped_feats)

In [12]:
rf = RandomForestClassifier()
rf.fit(X2, y2)
rf.score(X_test2, y_test2)

0.8020928352061566

In [13]:
with open('algos/RandomForestClassifier2.pickle', 'wb') as f:
    pickle.dump(rf, f)

In [ ]:
top_20 = np.argsort(rf.feature_importances_)[-1:-21:-1]
X_test2.columns[top_20]

In [ ]:
sorted(rf.feature_importances_)[-1:-21:-1]

In [16]:
lr = LogisticRegression()
lr.fit(X2, y2)
lr.score(X_test2, y_test2)

0.80352284631961346

In [17]:
with open('algos/LogRegressionClassifier2.pickle', 'wb') as f:
    pickle.dump(lr, f)

In [ ]:
top_20 = np.argsort(lr.coef_[0])[-1:-21:-1]
X_test2.columns[top_20]

In [ ]:
sorted(lr.coef_[0])[-1:-21:-1]

Last bit of exploration with the word space

In [10]:
train_w = [rev['words'] for rev in train2]

In [15]:
with open('algos/word_feats1.pickle', 'rb') as f:
    word_feats = pickle.load(f)

In [16]:
pre, post = exploratory_analysis(train_w, word_feats)

In [ ]:
# remove always (intensifier)
# remove "of", map "taste" to "food"
# leave "mouth" and "flavors" and "enjoyed"
# maybe leave "definitely" and "highly"

for w in ['definitely','highly','always','not always','enjoyed','mouth','taste','flavors']:
    print w.upper()
    print pre[w], '\n'
    print post[w], '\n'

- implement adverbs INCLUDING intensifiers
- portions: "the portions were __", huge, large, small, generous, big

horrible, regular, short, terrible, beautiful, entire, incredible, unique, wouldn't, awful, average, casual, interesting, fat, usual, delish, typical, avoid, standard, fair, exceptional
- flavorful food
- not flavorful food
- helpful service
- not helpful service
- mean service
- not mean service
- generous portion
- not generous portion
- creamy food
- not creamy food
- crunchy food
- not crunchy food
- 'portions': 'portion'
- 'cafe': 'restaurant'
- expensive (investigate)

In [ ]:
print [w for w in temp if 'RB' in w[1]]

In [ ]:
print [w for w in temp if 'NN' in w[1]]

Set up pipeline for generating component scores for each review

In [15]:
with open('algos/weights.pickle', 'rb') as f:
    weights = pickle.load(f)
with open('algos/food.pickle', 'rb') as f:
    food = pickle.load(f)
with open('algos/service.pickle', 'rb') as f:
    service = pickle.load(f)
with open('algos/atmosphere.pickle', 'rb') as f:
    atmosphere = pickle.load(f)
with open('algos/value.pickle', 'rb') as f:
    value = pickle.load(f)
with open('algos/decor.pickle', 'rb') as f:
    decor = pickle.load(f)

In [6]:
biz_id = 'ydata/akikos-restaurant-san-francisco'
with open(biz_id, 'r') as f:
    temp = sj.load(f)['text']
    biz_id = biz_id[6:]
temp = [''.join(re.split(r'<br/>|\n', doc)) for doc in temp]

In [8]:
filtered = ['really', 'very', 'even', 'much', 'real', 'customer', 'the', \
           'our', 'my', 'is', 'was']
subjects = ['service', 'ambiance', 'portions', 'food', 'decor', 'price', \
           'restaurant', 'experience']

In [9]:
data = []
for rev in temp:
    text = rev.lower().encode('ascii', 'ignore')
    words = nltk.word_tokenize(text)
    words = [w for w in words if w not in filtered]
    words = mapper(words)
    words = join_fol(words, ['not'])
    words = join_pre(words, ['priced'])
    words = re_position(words)
    words = join_pre(words, subjects)
    words = [w for w in words if 'restaurant' not in w and 'experience' not in w]
    data.append(words)

In [10]:
df = pd.DataFrame()
df['text'] = temp
df['words'] = data

In [13]:
with open('algos/word_feats1.pickle', 'rb') as f:
    word_feats = pickle.load(f)

In [16]:
for w in word_feats:
    df[w] = df.apply(lambda x: weights[w] if w in x.words else 0, axis=1)

In [140]:
df['food'] = df[food].sum(axis=1)
df['service'] = df[service].sum(axis=1)
df['atmosphere'] = df[atmosphere].sum(axis=1)
df['value'] = df[value].sum(axis=1)
df['decor'] = df[decor].sum(axis=1)

In [141]:
print biz_id
for component in ['food', 'service', 'atmosphere', 'value', 'decor']:
    print df[component].sum() / df.shape[0]
print ''

akikos-restaurant-san-francisco
0.668119402985
0.0793233830846
0.0391741293532
-0.0385870646766
-0.000338308457711



In [ ]:
# cd ydata

In [42]:
biz_weights('hog-island-oyster-co-san-francisco')

['hog-island-oyster-co-san-francisco', '4.6', '3.6', '3.3', '2.7', '3.0']

In [43]:
biz_weights('akikos-restaurant-san-francisco')

['akikos-restaurant-san-francisco', '5.0', '3.7', '3.9', '2.5', '3.0']

###Examine the Model by Making Searches

Food and value

In [1]:
import sqlite3
conn = sqlite3.connect('my_app/database/restaurants.db')
cur = conn.cursor()

In [221]:
CALL = '''SELECT id, food, value FROM sf ORDER BY 
            food*? + service*? + atmosphere*? + value*? + decor*? DESC'''
WEIGHTS = (5, 1, 1, 5, 1)
cur.execute(CALL, WEIGHTS)
cur.fetchmany(20)

[(u'delancey-street-restaurant-san-francisco', 4.1, 5.0),
 (u'front-door-cafe-san-francisco', 4.7, 4.4),
 (u'cafe-algiers-san-francisco', 4.3, 4.5),
 (u'chez-fayala-san-francisco', 4.8, 4.0),
 (u'dragoneats-san-francisco-3', 4.9, 3.9),
 (u'eatsa-san-francisco', 4.4, 4.4),
 (u'south-park-caf%C3%A9-san-francisco-4', 4.6, 3.7),
 (u'cafe-du-soleil-san-francisco-4', 5.0, 3.4),
 (u'la-fusi%C3%B3n-san-francisco-2', 4.8, 3.4),
 (u'barbacco-san-francisco', 4.6, 3.4),
 (u'eden-plaza-cafe-san-francisco', 4.4, 3.9),
 (u'crossroads-cafe-san-francisco-7', 3.9, 4.2),
 (u'fearless-coffee-san-francisco-3', 4.1, 3.9),
 (u'marlowe-san-francisco-2', 5.0, 2.9),
 (u'north-india-restaurant-san-francisco', 4.2, 3.4),
 (u'garaje-san-francisco', 4.5, 3.5),
 (u'sweet-joannas-cafe-san-francisco-2', 4.6, 3.5),
 (u'la-briciola-san-francisco', 4.5, 3.1),
 (u'portico-restaurant-san-francisco-3', 4.2, 3.9),
 (u'tropisue%C3%B1o-san-francisco-3', 4.5, 3.2)]

Food, service, atmosphere

In [220]:
CALL = '''SELECT id, food, service, atmosphere FROM sf ORDER BY 
            food*? + service*? + atmosphere*? + value*? + decor*? DESC'''
WEIGHTS = (5, 5, 5, 1, 1)
cur.execute(CALL, WEIGHTS)
cur.fetchmany(20)

[(u'north-india-restaurant-san-francisco', 4.2, 5.0, 4.6),
 (u'la-briciola-san-francisco', 4.5, 4.7, 4.2),
 (u'crepe-madame-san-francisco-2', 4.5, 4.7, 4.0),
 (u'marlowe-san-francisco-2', 5.0, 4.1, 4.0),
 (u'per-diem-san-francisco', 4.3, 3.6, 5.0),
 (u'barbacco-san-francisco', 4.6, 3.9, 4.4),
 (u'hops-and-hominy-san-francisco', 4.4, 4.1, 4.6),
 (u'zero-zero-san-francisco', 4.9, 3.8, 4.3),
 (u'sauce-san-francisco-3', 4.3, 3.8, 4.8),
 (u'pachino-trattoria-and-pizzeria-san-francisco', 4.7, 4.1, 3.9),
 (u'south-park-caf%C3%A9-san-francisco-4', 4.6, 3.5, 4.3),
 (u'akikos-restaurant-san-francisco', 5.0, 3.7, 3.9),
 (u'la-fusi%C3%B3n-san-francisco-2', 4.8, 4.1, 3.5),
 (u'delancey-street-restaurant-san-francisco', 4.1, 4.5, 3.5),
 (u'tropisue%C3%B1o-san-francisco-3', 4.5, 3.7, 4.1),
 (u'trace-san-francisco', 4.4, 4.4, 3.6),
 (u'fearless-coffee-san-francisco-3', 4.1, 4.9, 3.0),
 (u'front-door-cafe-san-francisco', 4.7, 4.2, 3.0),
 (u'local-kitchen-and-wine-merchant-san-francisco-2', 4.3, 3.9, 4.

Decor and atmosphere

In [225]:
CALL = '''SELECT id, atmosphere, decor FROM sf ORDER BY 
            food*? + service*? + atmosphere*? + value*? + decor*? DESC'''
WEIGHTS = (1, 1, 5, 1, 5)
cur.execute(CALL, WEIGHTS)
cur.fetchmany(20)

[(u'per-diem-san-francisco', 5.0, 3.8),
 (u'burritt-room-tavern-san-francisco', 4.8, 3.3),
 (u'sauce-san-francisco-3', 4.8, 3.0),
 (u'barbacco-san-francisco', 4.4, 3.0),
 (u'hops-and-hominy-san-francisco', 4.6, 3.0),
 (u'the-grove-yerba-buena-san-francisco', 4.6, 3.3),
 (u'north-india-restaurant-san-francisco', 4.6, 2.6),
 (u'la-capra-san-francisco-2', 4.3, 3.3),
 (u'south-park-caf%C3%A9-san-francisco-4', 4.3, 3.0),
 (u'trou-normand-san-francisco', 4.2, 3.5),
 (u'zero-zero-san-francisco', 4.3, 3.0),
 (u'la-briciola-san-francisco', 4.2, 2.9),
 (u'the-pink-elephant-san-francisco-3', 4.5, 3.0),
 (u'crepe-madame-san-francisco-2', 4.0, 3.0),
 (u'marlowe-san-francisco-2', 4.0, 3.0),
 (u'osha-thai-san-francisco', 4.4, 3.0),
 (u'tropisue%C3%B1o-san-francisco-3', 4.1, 3.0),
 (u'samovar-tea-lounge-san-francisco-2', 4.2, 3.3),
 (u'archive-bar-and-kitchen-san-francisco', 4.1, 3.0),
 (u'jersey-san-francisco', 3.0, 4.0)]

Fooooooood

In [218]:
CALL = '''SELECT id, food FROM sf ORDER BY 
            food*? + service*? + atmosphere*? + value*? + decor*? DESC'''
WEIGHTS = (5, 1, 1, 1, 1)
cur.execute(CALL, WEIGHTS)
cur.fetchmany(20)

[(u'marlowe-san-francisco-2', 5.0),
 (u'cafe-du-soleil-san-francisco-4', 5.0),
 (u'zero-zero-san-francisco', 4.9),
 (u'akikos-restaurant-san-francisco', 5.0),
 (u'front-door-cafe-san-francisco', 4.7),
 (u'la-fusi%C3%B3n-san-francisco-2', 4.8),
 (u'barbacco-san-francisco', 4.6),
 (u'dragoneats-san-francisco-3', 4.9),
 (u'south-park-caf%C3%A9-san-francisco-4', 4.6),
 (u'la-briciola-san-francisco', 4.5),
 (u'chez-fayala-san-francisco', 4.8),
 (u'crepe-madame-san-francisco-2', 4.5),
 (u'pachino-trattoria-and-pizzeria-san-francisco', 4.7),
 (u'per-diem-san-francisco', 4.3),
 (u'freshroll-vietnamese-rolls-and-bowls-san-francisco', 4.9),
 (u'north-india-restaurant-san-francisco', 4.2),
 (u'tropisue%C3%B1o-san-francisco-3', 4.5),
 (u'delancey-street-restaurant-san-francisco', 4.1),
 (u'wanna-e-san-francisco', 4.6),
 (u'super-duper-burgers-san-francisco-3', 4.7)]

Mix of everything

In [217]:
CALL = '''SELECT * FROM sf ORDER BY 
            food*? + service*? + atmosphere*? + value*? + decor*? DESC'''
WEIGHTS = (5, 5, 5, 5, 5)
cur.execute(CALL, WEIGHTS)
cur.fetchmany(20)

[(u'delancey-street-restaurant-san-francisco', 4.1, 4.5, 3.5, 5.0, 2.8),
 (u'north-india-restaurant-san-francisco', 4.2, 5.0, 4.6, 3.4, 2.6),
 (u'per-diem-san-francisco', 4.3, 3.6, 5.0, 2.9, 3.8),
 (u'la-briciola-san-francisco', 4.5, 4.7, 4.2, 3.1, 2.9),
 (u'barbacco-san-francisco', 4.6, 3.9, 4.4, 3.4, 3.0),
 (u'front-door-cafe-san-francisco', 4.7, 4.2, 3.0, 4.4, 3.0),
 (u'crepe-madame-san-francisco-2', 4.5, 4.7, 4.0, 3.0, 3.0),
 (u'south-park-caf%C3%A9-san-francisco-4', 4.6, 3.5, 4.3, 3.7, 3.0),
 (u'marlowe-san-francisco-2', 5.0, 4.1, 4.0, 2.9, 3.0),
 (u'fearless-coffee-san-francisco-3', 4.1, 4.9, 3.0, 3.9, 3.0),
 (u'la-fusi%C3%B3n-san-francisco-2', 4.8, 4.1, 3.5, 3.4, 3.0),
 (u'sauce-san-francisco-3', 4.3, 3.8, 4.8, 2.8, 3.0),
 (u'zero-zero-san-francisco', 4.9, 3.8, 4.3, 2.7, 3.0),
 (u'cafe-du-soleil-san-francisco-4', 5.0, 3.8, 3.0, 3.4, 3.3),
 (u'crossroads-cafe-san-francisco-7', 3.9, 3.5, 3.9, 4.2, 3.0),
 (u'hops-and-hominy-san-francisco', 4.4, 4.1, 4.6, 2.4, 3.0),
 (u'tropisue%C3%

In [216]:
CALL = '''SELECT * FROM sf ORDER BY 
            food*? + service*? + atmosphere*? + value*? + decor*? DESC'''
WEIGHTS = (5, 4, 2, 3, 1)
cur.execute(CALL, WEIGHTS)
cur.fetchmany(20)

[(u'delancey-street-restaurant-san-francisco', 4.1, 4.5, 3.5, 5.0, 2.8),
 (u'north-india-restaurant-san-francisco', 4.2, 5.0, 4.6, 3.4, 2.6),
 (u'front-door-cafe-san-francisco', 4.7, 4.2, 3.0, 4.4, 3.0),
 (u'la-briciola-san-francisco', 4.5, 4.7, 4.2, 3.1, 2.9),
 (u'crepe-madame-san-francisco-2', 4.5, 4.7, 4.0, 3.0, 3.0),
 (u'marlowe-san-francisco-2', 5.0, 4.1, 4.0, 2.9, 3.0),
 (u'fearless-coffee-san-francisco-3', 4.1, 4.9, 3.0, 3.9, 3.0),
 (u'barbacco-san-francisco', 4.6, 3.9, 4.4, 3.4, 3.0),
 (u'la-fusi%C3%B3n-san-francisco-2', 4.8, 4.1, 3.5, 3.4, 3.0),
 (u'cafe-du-soleil-san-francisco-4', 5.0, 3.8, 3.0, 3.4, 3.3),
 (u'south-park-caf%C3%A9-san-francisco-4', 4.6, 3.5, 4.3, 3.7, 3.0),
 (u'zero-zero-san-francisco', 4.9, 3.8, 4.3, 2.7, 3.0),
 (u'per-diem-san-francisco', 4.3, 3.6, 5.0, 2.9, 3.8),
 (u'wanna-e-san-francisco', 4.6, 4.3, 3.0, 3.0, 3.0),
 (u'pachino-trattoria-and-pizzeria-san-francisco', 4.7, 4.1, 3.9, 2.5, 3.0),
 (u'tropisue%C3%B1o-san-francisco-3', 4.5, 3.7, 4.1, 3.2, 3.0),
 

In [ ]:
conn.close()

In [ ]:
import matplotlib.pyplot as plt
%pylab inline

In [ ]:
['hog-island-oyster-co-san-francisco', '4.6', '3.6', '3.3', '2.7', '3.0']
fig = plt.figure()
ax = fig.add_subplot(111)

components = ['food', 'service', 'atmosphere', 'value', 'decor']
x_pos = np.arange(len(components))
scores = [4.6, 3.6, 3.3, 2.7, 3.0]

bars = plt.bar(x_pos, scores, width=0.5, alpha=0.9)
plt.xticks(x_pos, components, fontsize=16, rotation=45)
bars[0].set_color('#c41200')
bars[1].set_color('#e64518')
bars[2].set_color('#e64518')
bars[3].set_color('#e39f59')
bars[4].set_color('#e64518')
plt.title('Hog Island Oyster Co', fontsize=18)
ylim(ymax=5)
ax.grid(False);